In [ ]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("sonar-mine.csv")
df

In [ ]:
print(df.info())

In [ ]:
print(df.isnull().sum())

In [ ]:
le = LabelEncoder()
df['R'] = le.fit_transform(df['R'])

In [ ]:
X = df.drop(['R'], axis=1)
Y = df['R']

In [ ]:
X, Y = shuffle(X, Y, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(np.shape(X))

In [ ]:
x = tf.placeholder(tf.float32, [None, 60])
y = tf.placeholder(tf.float32, [None, 2])

In [ ]:
w = {"h1" : tf.Variable(tf.truncated_normal([60, 60])),
    "h2" : tf.Variable(tf.truncated_normal([60, 60])),
    "h3" : tf.Variable(tf.truncated_normal([60, 60])),
    "h4" : tf.Variable(tf.truncated_normal([60, 60])),
    "out" : tf.Variable(tf.truncated_normal([60, 2]))}

In [ ]:
b = {"b1" : tf.Variable(tf.truncated_normal([60])),
    "b2" : tf.Variable(tf.truncated_normal([60])),
    "b3" : tf.Variable(tf.truncated_normal([60])),
    "b4" : tf.Variable(tf.truncated_normal([60])),
    "out" : tf.Variable(tf.truncated_normal([2]))}

In [ ]:
layer1 = tf.nn.sigmoid(tf.add(tf.matmul(x, w["h1"]), b["b1"]))
layer2 = tf.nn.sigmoid(tf.add(tf.matmul(layer1, w["h2"]), b["b2"]))
layer3 = tf.nn.sigmoid(tf.add(tf.matmul(layer2, w["h3"]), b["b3"]))
layer4 = tf.nn.sigmoid(tf.add(tf.matmul(layer3, w["h4"]), b["b4"]))
y_predict = tf.add(tf.matmul(layer4, w["out"]), b["out"])

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predict, labels=y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
delta = tf.equal(tf.argmax(y, 1), tf.argmax(y_predict, 1))
accuracy = tf.reduce_mean(tf.cast(delta, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(1000) :
    sess.run(train, feed_dict = {x : x_train, y : y_train})
    loss_ = sess.run(loss, feed_dict = {x : x_train, y : y_train})
    accuracy_ = sess.run(accuracy, feed_dict = {x : x_train, y : y_train})
    print(f'epoch = {epoch} -> loss = {loss_} -> accuracy = {accuracy_}')

In [ ]:
loss_ = sess.run(loss, feed_dict = {x : x_test, y : y_test})
accuracy_ = sess.run(accuracy, feed_dict = {x : x_test, y : y_test})
print(loss_, accuracy_)
sess.close()